In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('final_data2-new.csv', index_col=0)
pd.set_option('display.max_colwidth', None)

In [3]:
df.label.value_counts()

label
pos    1404118
neg     217096
Name: count, dtype: int64

In [4]:
df['label_num'] = df.label.map({'pos':0, 'neg':1})
df

,tweet,label,label_num
0,globe with meridians new on the website globe with meridians this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,pos,0
1,they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,pos,0
2,sometimes i be think to myself when some of you post depress shit like what the hell you be depress about the scenario you make in your head nothing ever happen to you loudly crying faceface with tears of joyface with tears of joy,neg,1
3,let u get talk this week mental health awareness week talk to a friend colleague parent sibling or therapist can help url url,pos,0
4,user how can i best support you right now,pos,0
...,...,...,...
1621213,just woke up have no school be the best feeling ever,pos,0
1621214,thewdb com very cool to hear old walt interviews url,pos,0
1621215,be you ready for your mojo makeover ask me for detail,pos,0
1621216,happy 38th birthday to my boo of alll time tupac amaru shakur,pos,0


In [5]:
df = df.dropna(how='any',axis=0)
df[df['tweet'].isnull()]

,tweet,label,label_num


In [6]:
#defining X and y for use with COUNTVECTORIZER
X = df.tweet
y = df.label_num
print(X.shape)
print(y.shape)

(1621093,)
(1621093,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1215819,)
(405274,)
(1215819,)
(405274,)


In [8]:
X_train.describe()

count     1215819
unique    1160902
top          user
freq         2186
Name: tweet, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()

In [10]:
# only keep terms that appear in at least 2 documents
vect = CountVectorizer(min_df=2, max_df=0.5, stop_words='english', ngram_range=(1, 2))

In [11]:
vect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.5,
 'max_features': None,
 'min_df': 2,
 'ngram_range': (1, 2),
 'preprocessor': None,
 'stop_words': 'english',
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [12]:
# Document - term - matrix(dtm)
X_train_dtm = vect.fit_transform(X_train.values.astype('U'))

In [13]:
X_test_dtm = vect.transform(X_test.values.astype('U'))
X_test_dtm

<405274x681927 sparse matrix of type '<class 'numpy.int64'>'
	with 3804594 stored elements in Compressed Sparse Row format>

In [14]:
vect.vocabulary_

{'quot': 451697,
 'faulty': 182313,
 'penis': 421684,
 'freak': 200759,
 'lol': 328641,
 'freak lol': 200854,
 'lol quot': 329985,
 'user': 611674,
 'wala': 635033,
 'na': 381722,
 'lunch': 343909,
 'time': 572160,
 'stop': 533809,
 'serve': 497012,
 'breakfast': 74879,
 'food': 196862,
 'hahaha': 235054,
 'user wala': 624753,
 'wala na': 635039,
 'lunch time': 344402,
 'time na': 573738,
 'na stop': 381817,
 'stop serve': 534364,
 'serve breakfast': 497018,
 'breakfast food': 74999,
 'na hahaha': 381753,
 'mourn': 377024,
 'loss': 335471,
 'hamster': 237142,
 'rolly': 477636,
 'rip': 475494,
 'lt': 341991,
 'mourn loss': 377032,
 'rip lt': 475586,
 'luckyyyy': 343701,
 'suffer': 541276,
 'school': 490160,
 'tuesday': 593028,
 'user luckyyyy': 619087,
 'suffer school': 541347,
 'school tuesday': 491105,
 'hurt': 270056,
 'try': 590931,
 'yeah': 672922,
 'think': 564436,
 'understand': 603457,
 'like': 317014,
 'stupid': 538831,
 'fan': 180006,
 'user hurt': 617703,
 'hurt try': 270506,

In [15]:
# dict(sorted(vect.vocabulary_.items(), key=lambda item: item[1], reverse=True))
# np.asarray(X_train_dtm.sum(axis=0))


# counts = pd.DataFrame(X_train_dtm.toarray(),
#                       columns=vectorizer.get_feature_names())

# # Show us the top 10 most common words
# counts.T.sort_values(by=0, ascending=False).head(10)

In [16]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [17]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 142 ms, sys: 16.4 ms, total: 159 ms
Wall time: 160 ms


MultinomialNB()

In [18]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [19]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.925097588298287

In [20]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[340506,  10431],
       [ 19925,  34412]])

In [21]:
# print message text for the false positives (pos incorrectly classified as neg)
X_test[y_pred_class > y_test]

1142466                                                                    ahhh my room be a disaster if someone tell me how to twitpic i will
106782     my girlfriend be in italy for a week visit grandparents amaze for her but i miss her too much already i be like a lost little puppy
109585                                                                                     user it be not kinky unfortunately it be just weird
632363                                   everything i have ever have on my computer be deleted i legit think i be go to cry omg all my picture
173052                                                                user stupid busy lol i have be abandon etsy and the hooker how about you
                                                                          ...                                                                 
436385                                                                                                    missn my family like crazy right now

In [22]:
# print message text for the false negatives (neg incorrectly classified as pos)
X_test[y_pred_class < y_test]

225634                                                                                         be mad at library for not have all the book i want
1291399    one week of my quot sabbatical quot hop to have some sick stuff to take back to my producer new version of quot here quot to come soon
142836                   make it through the weekend i be really depress the last few days not excite about work though another week here we come
458208                                                                                      realllly really do not want to go to six form tomorow
1330331                                                user re no wordprocessor there be always the old pen and paper if you get desperate enough
                                                                            ...                                                                  
148005     get to get off ze net and study yes i know tragic favourite song atm be east jesus nowhere and horseshoe and hand

In [23]:
nb.predict_proba(X_test_dtm)

array([[0.24207125, 0.75792875],
       [0.99238903, 0.00761097],
       [0.86613221, 0.13386779],
       ...,
       [0.98785647, 0.01214353],
       [0.96453498, 0.03546502],
       [0.99375139, 0.00624861]])

In [24]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([0.75792875, 0.00761097, 0.13386779, ..., 0.01214353, 0.03546502,
       0.00624861])

In [25]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9112567330702188

In [26]:
# from sklearn.svm import SVC
# svm = SVC()

In [27]:
# # train the model using X_train_dtm
# %time svm.fit(X_train_dtm, y_train)
# # make class predictions for X_test_dtm
# y_pred_class = svm.predict(X_test_dtm)

In [28]:
# # calculate accuracy
# metrics.accuracy_score(y_test, y_pred_class)

In [29]:
# ============ #

In [30]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [31]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 1min 7s, sys: 30.6 s, total: 1min 38s
Wall time: 13.5 s


/Users/achintyajha/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [33]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([0.99372732, 0.00573321, 0.02737858, ..., 0.01470892, 0.02973203,
       0.01119032])

In [34]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.9522175121029229

In [35]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9590940448550436

In [36]:
# print message text for the false positives (pos incorrectly classified as neg)
X_test[y_pred_class > y_test]

109585                                                                                       user it be not kinky unfortunately it be just weird
1431260       user i know what you mean and you cannot even be 2 minute late quot oh so 2 minute ago i could have have pancakes but not now quot
417869                                 user it be because the damn stool be so low if i have the glass on the desk i would not have that problem
581976         i be feel really ill at the moment cannot go to the rest of retreat win but i need to get over this flu before my deb on saturday
526873     i have never felt so bad in my life i hurt tha one person that mean tha most to me nd i do not know why i did gahd i love him so much
                                                                           ...                                                                  
1290578                                                                                         be count down the day for this wee

In [37]:
# print message text for the false negatives (neg incorrectly classified as pos)
X_test[y_pred_class < y_test]

142836          make it through the weekend i be really depress the last few days not excite about work though another week here we come
458208                                                                             realllly really do not want to go to six form tomorow
207967                                           omg that be so boring why so serious danny they do not even ask him about scream on lol
1029965            user sorry to hear you could not find the help you needed maybe i could help you with with any of your user questions
644806                                                                           user do you ever find our terror watchdog i have no joy
                                                                       ...                                                              
577195     user mine do the same thing a few day ago and i be force to reinstall the firmware and i lose all my info damn you grey apple
1111697                i need to make a s

In [38]:
from preprocessing_universal import *

Hello, this function is the updated one. :)


In [39]:
input_text = input("Enter a tweet: ")
user_df = pd.DataFrame(columns=["text"], data=[[input_text]])
user_df = preprocessing(user_df)
X_user = user_df.tweet
X_user_dtm = vect.transform(X_user.values.astype('U'))
y_user_class = logreg.predict(X_user_dtm)
for i in y_user_class:
    if i==1:
        print("\nResult- Most likely depressed.")
        y_user_prob = logreg.predict_proba(X_user_dtm)[:, 1]
    elif i==0:
        print("\nResult- Most likely not depressed.")
        y_user_prob = logreg.predict_proba(X_user_dtm)[:, 0]
    else:
        print("unexpected error")
        
print(f"Probability: {y_user_prob}")

Enter a tweet:  asd



Result- Most likely not depressed.
Probability: [0.97281591]


In [40]:
user_df

,tweet,label
0,asd,pos


In [44]:
import pickle

# save
with open('model_d1_normal_logreg.pkl','wb') as f:
    pickle.dump(logreg,f)
    


In [45]:
with open('vectorizer_d1', 'wb') as f:
    pickle.dump(vect, f)

In [43]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/achintyajha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True